# Initialization

In [15]:
using LinearAlgebra
using JuMP
using GLPK
using Distributions
using Plots

# Problem 1 - Week 10

In [30]:
###
# x is duration parked
# cp is 1 DKK/min
# c+ is overtime at 3 DKK/min
# between 50 and 70 with uniform distribution


### Question 1
# It is a newsvendor problem
# SInce you have an uncertain amount of beers sold on a day
# and a cost for ordering too little and a saving by ordering too much

### Question 2
# Underage is when you order too much time and the cost must be 1 DKK/min per amount ordered over
# Overage cost is the 2 DKK/min because you ordered too little and pay the fee

### Question 3
# Time of the meeting is unceartain and is uniformly distributed

### Question 4
# Expected utility maximization slides 13 week 9
# (p-c)*(omega-x)+(c-z)*(x-omega)
# Pio = (p-c)
# Piu = (c-z)

xomega = [-20; 0; 20]
y = 2*xomega*(xomega >= 0) + (-1)*xomega*(xomega < 0)

p = plot(xomega,y)
@show p
# plots er umulige




### Question 5
# Use theorem (newsvendor problem) at slide 17 for week 9
#x = 2/(2+1)
#dist = Uniform(50,70)
#x_start = quantile(dist,x)



MethodError: MethodError: no method matching isless(::Int64, ::Vector{Int64})
Closest candidates are:
  isless(!Matched::AbstractVector, ::AbstractVector) at C:\Users\rasmu\AppData\Local\Programs\Julia-1.7.2\share\julia\base\abstractarray.jl:2532
  isless(::Integer, !Matched::ForwardDiff.Dual{Ty}) where Ty at C:\Users\rasmu\.julia\packages\ForwardDiff\PBzup\src\dual.jl:145
  isless(::Real, !Matched::AbstractFloat) at C:\Users\rasmu\AppData\Local\Programs\Julia-1.7.2\share\julia\base\operators.jl:185
  ...

# Week 11

In [4]:
#Import necessary Julia packages
using LinearAlgebra, JuMP, GLPK, Distributions, Random


In [ ]:
### Question 1
c1 = 200
c2 = 300
c3 = 40

n1 = 1
n2 = 1
n3 = 4

n_tot = 10

matrix = [[c1 c2 c3]; [10 300 0]; [250 550 5]]

In [17]:
## Question 1 and 2
ns = 5
pk = 1/5
omega = [40,45,50,70,100]
xv = []
ov = []


#Declare model and optimizer
rpmodel = Model()
set_optimizer(rpmodel, GLPK.Optimizer)

#Define variables
@variable(rpmodel, x1)
@variable(rpmodel, x2)
@variable(rpmodel, x3)
@variable(rpmodel, y1[1:5])
@variable(rpmodel, y2[1:5])
@variable(rpmodel, y3[1:5])
@variable(rpmodel, y4[1:5])
@variable(rpmodel, y5[1:5])

#Define Constraints
@constraint(rpmodel, x1 >= 10)
@constraint(rpmodel, x2 >= 10)
@constraint(rpmodel, x3 >= 40)
@constraint(rpmodel, [i=1:5], y2[i] >= 5)
@constraint(rpmodel, [i=1:5], y3[i]-y4[i] == 0)

@constraint(rpmodel,[i=1:5], x1+y1[i]+y3[i] == omega[i])
@constraint(rpmodel,[i=1:5], x2+y2[i]+y4[i] == omega[i])
@constraint(rpmodel,[i=1:5], x3+y5[i] == omega[i]*4)


@constraint(rpmodel,[i=1:5],  y1[i] >= 0)
@constraint(rpmodel,[i=1:5],  y2[i] >= 0)
@constraint(rpmodel,[i=1:5],  y3[i] >= 0)
@constraint(rpmodel,[i=1:5],  y4[i] >= 0)
@constraint(rpmodel,[i=1:5],  y5[i] >= 0)
@constraint(rpmodel, x1 >= 0)
@constraint(rpmodel, x2 >= 0)
@constraint(rpmodel, x3 >= 0)

#Define Objective
@objective(rpmodel, Min, x1*c1+x2*c2+x3*c3 + sum( ( 1/5* (10*y1[i]+300*y2[i]+250*y3[i]+550*y4[i]+5*y5[i]) ) for i in 1:5))
#Run the opimization
#Extract solutions

#Run the opimization
optimize!(rpmodel)

val = [value.(x1),value.(x2),value.(x3),value.(y1),value.(y2),value.(y3),value.(y4),value.(y5)]
ov = objective_value(rpmodel)

display(val)
display(ov)


8-element Vector{Any}:
 10.0
 35.0
 40.0
   [30.0, 35.0, 40.0, 60.0, 90.0]
   [5.0, 10.0, 15.0, 35.0, 65.0]
   [0.0, 0.0, 0.0, 0.0, 0.0]
   [0.0, 0.0, 0.0, 0.0, 0.0]
   [120.0, 140.0, 160.0, 240.0, 360.0]

23430.0

In [32]:
### Question 3
ns = 5
pk = 1/5
omega = [40,45,50,70,100]
xv = []
ov = []

for i in 1:ns
    #Declare model and optimizer
    wsmodel = Model()
    set_optimizer(wsmodel, GLPK.Optimizer)

    #Define variables
    @variable(wsmodel, x1)
    @variable(wsmodel, x2)
    @variable(wsmodel, x3)
    @variable(wsmodel, y1)
    @variable(wsmodel, y2)
    @variable(wsmodel, y3)
    @variable(wsmodel, y4)
    @variable(wsmodel, y5)

    #Define Constraints
    @constraint(wsmodel, x1 >= 10)
    @constraint(wsmodel, x2 >= 10)
    @constraint(wsmodel, x3 >= 40)
    @constraint(wsmodel, y2 >= 5)
    @constraint(wsmodel, y3-y4 == 0)

    @constraint(wsmodel, x1+y1+y3 == omega[i])
    @constraint(wsmodel, x2+y2+y4 == omega[i])
    @constraint(wsmodel, x3+y5 == omega[i]*4)


    @constraint(wsmodel, y1 >= 0)
    @constraint(wsmodel, y2 >= 0)
    @constraint(wsmodel, y3 >= 0)
    @constraint(wsmodel, y4 >= 0)
    @constraint(wsmodel, y5 >= 0)
    @constraint(wsmodel, x1 >= 0)
    @constraint(wsmodel, x2 >= 0)
    @constraint(wsmodel, x3 >= 0)

    #Define Objective
    @objective(wsmodel, Min, x1*c1+x2*c2+x3*c3 + (10*y1+300*y2+250*y3+550*y4+5*y5) )
    #Run the opimization
    #Extract solutions

    #Run the opimization
    optimize!(wsmodel)

    push!(xv,[value.(x1),value.(x2),value.(x3),value.(y1),value.(y2),value.(y3),value.(y4),value.(y5)])
    push!(ov,objective_value(wsmodel))
end
display(mean(xv))
display(mean(ov))


8-element Vector{Float64}:
  10.0
  56.0
  40.0
  51.0
   5.0
   0.0
   0.0
 204.0

23430.0

In [33]:
omega = [40,45,50,70,100]

evmodel = Model()
set_optimizer(evmodel, GLPK.Optimizer)

#Define variables
@variable(evmodel, x1)
@variable(evmodel, x2)
@variable(evmodel, x3)
@variable(evmodel, y1)
@variable(evmodel, y2)
@variable(evmodel, y3)
@variable(evmodel, y4)
@variable(evmodel, y5)

#Define Constraints
@constraint(evmodel, x1 >= 10)
@constraint(evmodel, x2 >= 10)
@constraint(evmodel, x3 >= 40)
@constraint(evmodel, y2 >= 5)
@constraint(evmodel, y3-y4 == 0)

@constraint(evmodel, x1+y1+y3 == mean(omega))
@constraint(evmodel, x2+y2+y4 == mean(omega))
@constraint(evmodel, x3+y5 == mean(omega)*4)


@constraint(evmodel, y1 >= 0)
@constraint(evmodel, y2 >= 0)
@constraint(evmodel, y3 >= 0)
@constraint(evmodel, y4 >= 0)
@constraint(evmodel, y5 >= 0)
@constraint(evmodel, x1 >= 0)
@constraint(evmodel, x2 >= 0)
@constraint(evmodel, x3 >= 0)

#Define Objective
@objective(evmodel, Min, x1*c1+x2*c2+x3*c3 + (10*y1+300*y2+250*y3+550*y4+5*y5) )
#Run the opimization
#Extract solutions

#Run the opimization
optimize!(evmodel)

val = [value.(x1),value.(x2),value.(x3),value.(y1),value.(y2),value.(y3),value.(y4),value.(y5)]
ov = objective_value(evmodel)

display(val)
println(ov)

8-element Vector{Float64}:
  10.0
  56.0
  40.0
  51.0
   5.0
   0.0
   0.0
 204.0

23430.0
